In [2]:
import sys
from pathlib import Path

# Get the project root 
project_root = Path.cwd().parent

# Add project root to sys.path
sys.path.append(str(project_root))

In [3]:
import pandas as pd
from src.preprocessing import (
    load_raw_2015,
    build_and_save_preprocessed,  
)

In [4]:
df_raw = load_raw_2015()
df_raw.shape

FileNotFoundError: File not found: E:\DataPrepPrj\data\raw\2015_raw.csv

In [ ]:
df_pre = build_and_save_preprocessed(threshold=0.5)


=== High-missing columns (> 0.5): ===
['_CHISPNC', 'JOINPAIN', 'ARTHDIS2', '_PNEUMO2', 'ARTHSOCL', 'LMTJOIN3', '_FLSHOT6', 'TRNSGNDR', 'BPMEDS', 'FLSHTMY2', 'SMOKDAY2', 'IMFVPLAC', 'PADUR2_', 'PAFREQ2_', 'MAXDRNKS', 'AVEDRNK2', 'DRNK3GE5']
Dropping 17 columns:
  - _CHISPNC
  - JOINPAIN
  - ARTHDIS2
  - _PNEUMO2
  - ARTHSOCL
  - LMTJOIN3
  - _FLSHOT6
  - TRNSGNDR
  - BPMEDS
  - FLSHTMY2
  - SMOKDAY2
  - IMFVPLAC
  - PADUR2_
  - PAFREQ2_
  - MAXDRNKS
  - AVEDRNK2
  - DRNK3GE5
Saved cleaned dataset to: E:\DataPrepPrj\data\processed\clean_2015.csv
